# HA 3

### Problem 1 (1 point)

Diffie–Hellman key exchange protocol is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key. 

1) Implement function to generate common secret key within multiplicative group of given Finite field with known generator. 

*Note. You can assume that all the numbers are small, for example, less than $2^{16}$.*

In [18]:
def get_shared_secret(p: int, my_private_key: int, other_public_key: int) -> int:
    """
    Returns the shared secret for the Diffie-Hellman key exchange protocol using own `my_private_key` and other's `other_public_key` 
    within multiplicative group of given finite field of order `p`.
    `p` is assumed to be a prime number and must be greater than 1.
    """
    assert p > 1, "p must be greater than 1"
    # Python already has an effective built-in function for modular exponentiation, so we use it. 
    return pow(other_public_key, my_private_key, p)

2) Test your solution in GF(17) with generator g=11. Bobs' open key B=11, Alice private key is a=7

In [19]:
# It'd be nice to show that Alice's and Bob's shared secret match.
# p=17 and g=11 are public parameters known to both parties.

alice_shared_secret = get_shared_secret(17, 7, 11)
print(f"Alice's shared secret: {alice_shared_secret}")

# Bob's public key is 11, so we have: 11 = 11^b mod 17, where b is Bob's private key.
# In reality, b should be hard to compute (discrete logarithm problem), 
# but in this case for such small numbers the solution to b is trivial: b = 1.
bob_private_key = 1

# Knowing Alice's private key we can compute her public key.
alice_public_key = pow(11, 7, 17)

# Finally, we can compute Bob's shared secret.
bob_shared_secret = get_shared_secret(17, bob_private_key, alice_public_key)
print(f"Bob's shared secret: {bob_shared_secret}")

Alice's shared secret: 3
Bob's shared secret: 3


As expected, shared secret of both parties match.

### Problem 2 (3 points)

El Gamal protocol is widely used in cryptography. In this task we will ask you to implement your own El-Gamal encryption scheme on Python.

1) Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed within the lectures). (1 point)

*Note. You can assume that all the numbers are small, for example, less than $2^{32}$. But you **must** use a primality test as a part of the function to get a full score.*

In [18]:
from secrets import randbelow # cryptographically secure

In [19]:
# A straightforward approach, definitely can be optimised.


def factorize(x: int) -> list[int]:
    """Yields prime factors of `x`."""
    # Simple trial division, works well enough for numbers smaller than 2**32.
    assert x >= 0, "x must be non-negative"
    factors = []
    f = 2
    while f * f <= x:
        if x % f == 0:
            if f not in factors:
                factors.append(f)
            x //= f
        else:
            f += 1
    if x > 1 and x not in factors:
        factors.append(x)
    
    return factors


def is_prime(x: int) -> bool:
    # Simple trial division, works well enough for numbers smaller than 2**32.
    assert x >= 0, "x must be non-negative"
    if x == 2:
        return True
    if x < 2 or x % 2 == 0:
        return False
    
    d = 3
    while d * d <= x:
        if x % d == 0:
            return False
        d += 2

    return True


def get_random_prime() -> int:
    """Generates a random uniformly distributed prime number in range [2**20, 2**32]."""
    # Naive approach, but should work well enough for our purposes.
    # From prime number theory we can estimate that there are about 2**32/ln(2**32) - 2**20/ln(2**20) ~ 193559611 prime numbers in that range.
    # That means the probability of generating a prime number is about 193559611/(2**32 - 2**20) ~ 4.5%.
    # Thus we expect the following loop to run about ~22 times on average, which is fine for our purposes.
    while True:
        p = randbelow((1<<32) - (1<<20)) + 1<<20
        # A little hack to make sure we get an odd number.
        # This shouldn't affect the uniform distribution for primality test, since almost all prime numbers are odd.
        if p & 1 == 0:
            p += 1
        if is_prime(p):
            return p
        

def get_group_generator(p: int) -> int:
    """
    Returns a random group generator of the multiplicative group of integers modulo `p`.
    
    `p` is assumed to be prime.
    """
    fs = factorize(p - 1)
    while True:
        # Shifting by 2 to avoid 0's and 1's.
        g = randbelow(p - 2) + 2
        # Essentially checking whether all elements can be generated by `g`.
        if all(pow(g, (p - 1) // f, p) != 1 for f in fs):
            return g
        

def get_private_key(p: int) -> int:
    """Returns a random private key in range [1, `p-2`] as element of multiplicative group of integers modulo `p`."""
    return randbelow(p - 2) + 1


PublicKey = tuple[int, int, int]


def get_public_key(p: int, g: int, private_key: int) -> PublicKey:
    """Returns a public key composed of a group modulo, a group generator, and a private key."""
    # Note that this function is deterministic -- given the same inputs, it will always return the same output.
    public_key = pow(g, private_key, p)
    return p, g, public_key

2) Implement functions that realize the encryption and decryption in El Gamal protocol. (1 points)

In [22]:
CipherText = tuple[int, int]


def encrypt(x: int, public_key: PublicKey) -> CipherText:
    """Encrypts given `x` using public key and produces an ephemeral public key and the encrypted message."""
    p, g, actual_public_key = public_key
    assert 0 <= x < p, "x must be in range [0, p)"

    ephemeral_key = get_private_key(p)
    ephemeral_public_key = pow(g, ephemeral_key, p)

    shared_secret = pow(actual_public_key, ephemeral_key, p)
    encrypted_message = (x * shared_secret) % p
    return ephemeral_public_key, encrypted_message


def decrypt(cipher_text: CipherText, private_key: int, p: int) -> int:
    """Decrypts given cipher text using `private_key` and group modulo `p`."""
    ephemeral_public_key, encrypted_message = cipher_text
    shared_secret = pow(ephemeral_public_key, private_key, p)
    decrypted_message = (encrypted_message * pow(shared_secret, -1, p)) % p
    return decrypted_message

3) Calculate Hash of your name by SHA-1 and test El Gamal encryption/decryption functions on its 16-bit prefix. (1 points)

In [23]:
from hashlib import sha1

In [25]:
p = get_random_prime()
g = get_group_generator(p)
private_key = get_private_key(p)
public_key = get_public_key(p, g, private_key)

print(f"Group modulo: {p}")
print(f"Group generator: {g}")
print(f"Private key: {private_key}")
print(f"Public key: {public_key}")

name = "Maxim"
hashed_name = sha1(name.encode()).digest()
message = int.from_bytes(hashed_name[:2])
print(f"Message: {message}")

cipher_text = encrypt(message, public_key)
print(f"Cipher text: {cipher_text}")

decrypted_message = decrypt(cipher_text, private_key, p)
print(f"Decrypted message: {decrypted_message}")

Group modulo: 3717953326415873
Group generator: 1511409967003651
Private key: 1694551960099017
Public key: (3717953326415873, 1511409967003651, 1254785817097861)
Message: 1977
Cipher text: (1805725999797894, 271152679143842)
Decrypted message: 1977


As expected, original and decrypted messages match.

### Problem 3 (4 points)

Elliptic curves due to their efficient hardware realization widely used in modern secure communication channels. The main thing that lies inside their cryptographic hardness is that we can break them only by greed search over all group points. In this task, we will ask you to write Python function that returns all group elements of a certain elliptic curve over a finite field 

1) Write a python function that list all points of elliptic curve $y^2=x^3+7$ over $F_{127}$ (1 point)

*Note. $127 = 2^7-1$ is the fourth Mersenne prime.*

In [ ]:
# We can simply brute-force the solutions since it's possible number (2**7-1)**2 ~ 2**14 is really small.
def get_ec_points(a: int, b: int, p: int) -> list[tuple[int, int]]:
    """Yields all points on the elliptic curve y^2 = x^3 + ax + b modulo `p`."""
    points = []
    for x in range(p):
        y_sq = (x**3 + a*x + b) % p
        for y in range(p):
            if (y*y) % p == y_sq:
                points.append((x, y))

    return points

In [11]:
# Elliptic curves are of form: y^2 = x^3 + ax + b
# In our case, we have: a = 0 and b = 7 over the field of integers modulo 127.
ec_solutions = get_ec_points(0, 7, 127)
print(ec_solutions)

[(1, 32), (1, 95), (2, 53), (2, 74), (3, 62), (3, 65), (4, 43), (4, 84), (8, 30), (8, 97), (11, 24), (11, 103), (12, 46), (12, 81), (14, 46), (14, 81), (17, 27), (17, 100), (18, 39), (18, 88), (19, 32), (19, 95), (21, 39), (21, 88), (24, 49), (24, 78), (25, 30), (25, 97), (28, 49), (28, 78), (32, 3), (32, 124), (34, 24), (34, 103), (38, 53), (38, 74), (39, 12), (39, 115), (41, 27), (41, 100), (45, 33), (45, 94), (46, 51), (46, 76), (47, 43), (47, 84), (51, 18), (51, 109), (52, 36), (52, 91), (57, 62), (57, 65), (58, 38), (58, 89), (60, 19), (60, 108), (67, 62), (67, 65), (69, 27), (69, 100), (70, 19), (70, 108), (71, 63), (71, 64), (72, 16), (72, 111), (75, 49), (75, 78), (76, 43), (76, 84), (78, 50), (78, 77), (79, 63), (79, 64), (80, 18), (80, 109), (82, 24), (82, 103), (84, 16), (84, 111), (85, 50), (85, 77), (86, 38), (86, 89), (87, 53), (87, 74), (88, 39), (88, 88), (91, 50), (91, 77), (93, 33), (93, 94), (94, 30), (94, 97), (96, 51), (96, 76), (98, 16), (98, 111), (99, 36), (99, 

2) Compare the number of points with Hasse’s estimate $|N-(q+1)|\leq 2{\sqrt  {q}}$. (1 point)

In [14]:
# Hasse's theormes bounds the number of solutions N using the the field's order q.
# From the inequality both bounds for N immediately follow:
# N - (q + 1) >= -2*sqrt(q) and N - (q + 1) <= 2*sqrt(q) <=> N >= q + 1 - 2*sqrt(q) and N <= q + 1 + 2*sqrt(q).
# Thus N must lie in range [q + 1 - 2*sqrt(q), q + 1 + 2*sqrt(q)].
q = 127
solutions_n_lower_bound = q + 1 - 2*q**0.5
solutions_n_upper_bound = q + 1 + 2*q**0.5
print(f"Hasse's estimate: {solutions_n_lower_bound:.3f} <= {len(ec_solutions)} <= {solutions_n_upper_bound:.3f}")

Hasse's estimate: 105.461 <= 126 <= 150.539


Computed number of solution bounds estimate seems right.

3) Prove that the point
$A = (19, 32)$ belongs to the elliptic curve and construct a sequence of $B_n = nA, n = 1, ..., 100$. (2 points)

In [ ]:
# To prove that the point A = (19, 32) belongs to the elliptic curve, we cans simply substitute its coordinates into the equation.
x, y = (19, 32)
lhs = pow(y, 2, 127)
rhs = (pow(x, 3, 127) + 7) % 127
print(f"{y}**2 = {x}**3 + 7 (mod 127)? {lhs == rhs}")

32**2 = 19**3 + 7 (mod 127)? True


In [17]:
# Using strightforward repeated addition.
bn = [(19, 32)]
for _ in range(1, 100):
    x, y = bn[-1]
    # Tangent to the elliptic curve at the current point:
    # 3*x**2 + a / 2*y
    slope = (3*x**2) * pow(2*y, -1, 127)

    # The rest is similar to regular point addition.
    x_new = (slope**2 - 2*x) % 127
    y_new = (slope*(x - x_new) - y) % 127

    bn.append((x_new, y_new))

print(bn)

[(19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 32), (11, 24), (123, 18), (38, 53), (122, 3), (71, 63), (85, 77), (19, 

### Problem 4 (2 points)

Let $p = 601$, $q = 7$, $e = 1463$ be the setup of the RSA algorithm. Compute $d$, sign the plane message $m = 58$ and check the signature.

### Problem 5 (4* points)

*Bonus Problem*

The dealer shared the secret among four participants using a (n, 4)-threshold Shamir secret sharing scheme over ( F_{19} ) with the following shares: ( (3, 17), (17, 5), (5, 11), (11, 4) ).

1) (2 points) Find the secret (the value at ( x=0 )) and the initial polynomial. Is it consistent with the (4, 3) threshold?

***Hint.*** *Refer to whiteboard 8.3 for an example.*

2) (2 points) Provide a probabilistic secret generation procedure for the aforementioned example and theoretically prove that it ensures perfect secrecy.